In [ ]:
%load_ext autoreload
%autoreload 2

In [10]:
from longeval.spark import get_spark
from pyspark.sql import functions as F

spark = get_spark()

24/12/24 14:17:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
df = spark.read.parquet("../../data/longeval/tokens/*/*/*").cache()
df.show()

+--------+-------+-----+----------+---------------+------+-----+
|language|   date|split|collection|             id|tokens|words|
+--------+-------+-----+----------+---------------+------+-----+
|  French|2023_08| test| documents|doc082302214482|   379|  176|
|  French|2023_08| test| documents|doc082302205059|  1909| 1183|
|  French|2023_08| test| documents|doc082302200515|   264|  168|
|  French|2023_08| test| documents|doc082302211708|     0|    1|
|  French|2023_08| test| documents|doc082302204885|  1542| 1151|
|  French|2023_08| test| documents|doc082301205404|   374|  232|
|  French|2023_08| test| documents|doc082302217498|   830|  490|
|  French|2023_08| test| documents|doc082301218918|     0|    1|
|  French|2023_08| test| documents|doc082302204786|  1941| 1546|
|  French|2023_08| test| documents|doc082301209434|  1156|  928|
|  French|2023_08| test| documents|doc082301200421|     0|    1|
|  French|2023_08| test| documents|doc082301200369|     0|    1|
|  French|2023_08| test| 

In [18]:
(
    df.where(F.col("collection") == "documents")
    .groupBy("split", "language", "collection", "date")
    .agg(
        F.sum("tokens").alias("sum_tokens"),
        F.sum("words").alias("sum_words"),
    )
    # divided by 1m
    .withColumn("sum_tokens_pm", F.col("sum_tokens") / 1_000_000)
    .withColumn("sum_words_pm", F.col("sum_words") / 1_000_000)
).show()

(
    df.where(F.col("collection") == "documents")
    .groupBy("split", "language")
    .agg(
        F.sum("tokens").alias("sum_tokens"),
        F.sum("words").alias("sum_words"),
    )
    # divided by 1m
    .withColumn("sum_tokens_pm", F.col("sum_tokens") / 1_000_000)
    .withColumn("sum_words_pm", F.col("sum_words") / 1_000_000)
).show()

+-----+--------+----------+-------+-------------+----------+-------------+------------+
|split|language|collection|   date|   sum_tokens| sum_words|sum_tokens_pm|sum_words_pm|
+-----+--------+----------+-------+-------------+----------+-------------+------------+
| test|  French| documents|2023_08|1.761001846E9|1130451326|  1761.001846| 1130.451326|
| test| English| documents|2023_08|1.648139339E9| 932733087|  1648.139339|  932.733087|
|train|  French| documents|2023_01|2.557447888E9|1640616023|  2557.447888| 1640.616023|
|train| English| documents|2023_01|2.375897725E9|1344163168|  2375.897725| 1344.163168|
| test|  French| documents|2023_06|1.815450509E9|1163542559|  1815.450509| 1163.542559|
| test| English| documents|2023_06|1.693284689E9| 960832171|  1693.284689|  960.832171|
+-----+--------+----------+-------+-------------+----------+-------------+------------+

+-----+--------+-------------+----------+-------------+------------+
|split|language|   sum_tokens| sum_words|sum_token

For OpenAI embedding models: https://openai.com/api/pricing/

| Model                | Pricing          | Pricing with Batch API* |
|----------------------|------------------|-------------------------|
| text-embedding-3-small | $0.020 / 1M tokens | $0.010 / 1M tokens      |
| text-embedding-3-large | $0.130 / 1M tokens | $0.065 / 1M tokens      |
| ada v2               | $0.100 / 1M tokens | $0.050 / 1M tokens      |

For GCP: https://cloud.google.com/vertex-ai/generative-ai/pricing

| Model               | Type  | Region | Price per 1,000 characters |
|---------------------|-------|--------|----------------------------|
| Embeddings for Text | Input | Global |                            |
|                     |       |        | Online requests: $0.000025 |
|                     |       |        | Batch requests: $0.00002   |

For every million characters:

| Model               | Type  | Region | Price per 1 million characters |
|---------------------|-------|--------|----------------------------|
| Embeddings for Text | Input | Global |                            |
|                     |       |        | Online requests: $25       |
|                     |       |        | Batch requests: $20        |

